In [1]:
import cufflinks as cf  # noqa
import numpy as np
import pandas as pd
import plotly.offline  # noqa
import statsmodels.api as sm
import scipy

In [2]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True, theme='white')

In [3]:
df = pd.read_excel('data/table.xlsx', usecols=[1, 2], names=['y', 'x'])
df

,y,x
0,310,390
1,350,385
2,330,545
3,425,680
4,502,810
5,360,780
6,420,790
7,505,785
8,280,400
9,305,530


# Парная регрессия

## Построение и расчет модели

#### Регрессионная модель

$$ y_i = a + b_1 x_{1i} + e_i $$

#### Оценочная модель

$$ \hat{y_i} = \hat{a} + \hat{b_1} x_{1i} $$

#### Результативный признак

In [4]:
y = df['y']
y

0     310
1     350
2     330
3     425
4     502
5     360
6     420
7     505
8     280
9     305
10    340
11    460
12    440
13    415
14    345
15    405
16    450
17    515
18    390
19    370
20    435
21    458
22    490
23    485
Name: y, dtype: int64

#### Фактор модели

In [5]:
import warnings

warnings.filterwarnings('ignore')

x = df['x']
x = sm.add_constant(x)
x

,const,x
0,1.0,390
1,1.0,385
2,1.0,545
3,1.0,680
4,1.0,810
5,1.0,780
6,1.0,790
7,1.0,785
8,1.0,400
9,1.0,530


#### Парная корреляция

In [6]:
x['x'].corr(y)

0.820219682079618

In [7]:
df.iplot(
    x='x', y='y',
    xTitle=f'x',
    yTitle=f'y',
    kind='scatter',
    mode='markers',
    size=8
)

#### Применение метода наименьших квадратов

$$ B = (X^T X)^{-1} X^T Y $$

In [8]:
X = np.matrix(x.to_numpy())
Y = np.matrix(y.to_numpy()).T
B = np.linalg.inv((X.T.dot(X))).dot(X.T).dot(Y)
B

matrix([[132.15465835],
        [  0.41856254]])

$$ \sum_{i}^{n}{(y_i - \hat{y_i})^2} \rightarrow \min_{n} $$

In [9]:
model = sm.OLS(y, x).fit()

In [10]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     45.23
Date:                Sat, 02 Oct 2021   Prob (F-statistic):           9.27e-07
Time:                        14:53:51   Log-Likelihood:                -121.84
No. Observations:                  24   AIC:                             247.7
Df Residuals:                      22   BIC:                             250.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        132.1547     41.799      3.162      0.0

#### Найденные коэффициенты

$$ \hat{y_i} = 132.1547 + 0.4186 x_{1i} $$

#### Оценки среднеквадратического отклонения оценок параметров

$S_{\hat{a}} = 41.799$

$S_{\hat{b_1}} = 0.062$

## Оценка качества модели

**Оценка среднеквадратического отклонения возмущений:** $ S_{ei} = 40.499 $ - **умеренная**.

In [11]:
np.sqrt(model.mse_resid)

40.49900301912734

**Коэффициент детерминации:** $R^2 = 0.673$ - доля дисперсии $y$,
объясняемая текущей моделью (т.е. объясняющими переменными).

$ R^2 > 0.5 $ - качество модели **ну такое~**.

**Средняя относительная ошибка аппроксимации:** $A = 8.396$.

$ 5\%< A < 10\% $ - **хорошая** оценка

In [12]:
np.mean(np.abs(model.resid / y)) * 100

8.396351371527706

## Оценка модели на значимость

**F-критерий Фишера:**

$$ H_0: b_1 = ... = b_k = 0, \\ H_1: b^2_1 + ... + b^2_k > 0. $$

$$ F_{набл} = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$

$F = 45.23$,

$p\text{-}value = 9.27e\text{-}07 $

$p\text{-}value < \alpha\{0.1;0.05;0.01\} \rightarrow$ отвергаем гипотезу $H_0$, т.е. модель регрессии в целом **значима**.

In [13]:
# F-критерий Фишера
# для проверки значимости модели
# F_табл < F_набл - модель в целом ЗНАЧИМА
a = 0.05  # уровень значимости
f_table = scipy.stats.f.ppf(1 - a, model.nobs, model.df_resid)  # noqa
f_table

2.0283185080245336

## Оценка значимости параметров модели

**t-критерий Стьюдета для оценки значимости параметров модели:**

$$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $$

$$ t_{b_i} = \frac{\hat{b_i} - 0}{\hat{\sigma_{b_i}}} $$

$t_a = 3.162$, $p\text{-}value = 0.005$, $p\text{-}value < \alpha\{0.1;0.05;0.01\} \rightarrow$
гипотеза $H_0$ отвергается - параметр $a$ **значим**.

$t_{b_1} = 6.725$, $p\text{-}value = 0.000$, $p\text{-}value < \alpha\{0.1;0.05;0.01\} \rightarrow$
гипотеза $H_0$ отвергается - параметр $b_1$ **значим**.

In [14]:
# t-критерий Стьюдента
# для оценки значимости параметров модели
# t_табл < t_набл - коэффициент b ЗНАЧИМО ОТЛИЧАЕТСЯ от [0].
a = 0.05  # уровень значимости
t_table = scipy.stats.t.ppf(1 - a / 2, model.df_resid)  # noqa
t_table

2.0738730679040147

**Доверительные интервалы:**

$$ \hat{b_i} - \hat{\sigma} \cdot t_{табл} \leq b_i \leq \hat{b_i} + \hat{\sigma} \cdot t_{табл} $$

Для **95%**:

$a: (45.470; 218.840)$

$b_1: (0.289; 0.548)$

In [15]:
pd.concat([(model.params - model.bse * t_table).rename('<'),
           (model.params + model.bse * t_table).rename('<')], axis=1)

,<,<
const,45.469585,218.839731
x,0.289490,0.547635


In [51]:
df.iplot(
    x='x', y='y',
    xTitle=f'x',
    yTitle=f'y',
    kind='scatter',
    mode='markers',
    size=8,
    bestfit=True
)

## Интерпретация параметров

Линейная зависимость **потребительские расходы**:
- коэффициент 0.4186 означает, что каждая единица **средней заработной платы**
  увеличивает **потребительские расходы** на 418.6 рублей.
- при отсутствии **заработной платы и социальных выплат**
  **потребительские расходы** сохранятся на уровне 132 тыс. руб.
